import ipy_autoreload
%autoreload 2

In [ ]:
import os, pandas as pd, pickle
from datetime import datetime, timedelta
# print (os.getcwd())

batch_day_count = 1000
industry = 'beauty'
today = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)

try:    
    indexDt = pickle.load(open('indexDt.'+industry, 'rb'))
except:
    indexDt = datetime.strptime('2018-01-23', '%Y-%m-%d') # fixed min DT

dates = list (pd.date_range(indexDt, today)) # datetimeIndex to list of Timestamp class

In [7]:
indexDt

datetime.datetime(2018, 1, 23, 0, 0)

In [19]:
dates = dates[0:37]
dates

[Timestamp('2018-01-23 00:00:00', freq='D'),
 Timestamp('2018-01-24 00:00:00', freq='D'),
 Timestamp('2018-01-25 00:00:00', freq='D'),
 Timestamp('2018-01-26 00:00:00', freq='D'),
 Timestamp('2018-01-27 00:00:00', freq='D'),
 Timestamp('2018-01-28 00:00:00', freq='D'),
 Timestamp('2018-01-29 00:00:00', freq='D'),
 Timestamp('2018-01-30 00:00:00', freq='D'),
 Timestamp('2018-01-31 00:00:00', freq='D'),
 Timestamp('2018-02-01 00:00:00', freq='D'),
 Timestamp('2018-02-02 00:00:00', freq='D'),
 Timestamp('2018-02-03 00:00:00', freq='D'),
 Timestamp('2018-02-04 00:00:00', freq='D'),
 Timestamp('2018-02-05 00:00:00', freq='D'),
 Timestamp('2018-02-06 00:00:00', freq='D'),
 Timestamp('2018-02-07 00:00:00', freq='D'),
 Timestamp('2018-02-08 00:00:00', freq='D'),
 Timestamp('2018-02-09 00:00:00', freq='D'),
 Timestamp('2018-02-10 00:00:00', freq='D'),
 Timestamp('2018-02-11 00:00:00', freq='D'),
 Timestamp('2018-02-12 00:00:00', freq='D'),
 Timestamp('2018-02-13 00:00:00', freq='D'),
 Timestamp

In [9]:
len(dates)*0.2

7.800000000000001

In [20]:
from ci_db import get_ci_revisit_regDt, get_ci_company_by_id, get_ci_adid,revisit_in_dt_df_write,filter_industry,load_snapshot

In [21]:
start_time =datetime.now()
#dates=dates[15:17]

for index, day in enumerate(dates):
    
    print('=======processing :',round((index+1) / len(dates)*100),'%')
    #in_companies와 included_companyid, in_users와 included_userid  각각 같은 파일에서 나온다
    in_revisits, in_users, in_companies, all_userid, included_userid, included_companyid, nonincluded_userid, nonincluded_companyid = load_snapshot(industry)
    
    try:
        #1) revisit_day : "temp variable"
        #03.02 get_ci_revisit_regDt() query문 수정
        revisit_day = get_ci_revisit_regDt(day)[['EPOPCON_ID','COMPANY_ID','DEAL_DT','PAYMENT','CARD_NAME','ISSUED_COMPANY_ID', 'REG_DT']]

        #1-2) company_day : "temp variable"  == revisit_day의 compid 
        compid_day = list(set(revisit_day.COMPANY_ID.unique().tolist())- set(included_companyid+nonincluded_companyid))
        companies_day = get_ci_company_by_id(compid_day)[['ID', 'CO_NAME', 'CO_NAME_R', 'CATE', 'CATE1', 'TAG']]
        companies_day = companies_day[companies_day['CO_NAME_R'].notnull()]
        companies_day['COMPANY_ID'] = companies_day['ID'].astype(int)
        del companies_day['ID']    

        #1-3) all_userid : revisit_day의 userid  추가하기
        all_userid = list(set(revisit_day.EPOPCON_ID.unique().tolist() + all_userid)) 
        pickle.dump(all_userid, open('user_all.'+industry, 'wb'))
    except:
        print(d, 'no revisit_day or no company_id')
        pickle.dump(d + timedelta(days=1), open('indexDt.'+industry, 'wb'))
        continue    
    
    # 2) filtering : by using keyword : company_day -> included or not
    # in_companies_day, nonincluded_compid_day  : "temp variable" 
    in_companies_day, nonincluded_compid_day = filter_industry(companies_day, industry)
    
    # 2-1) nonincluded_companyid 추가하기
    nonincluded_companyid = list(set ( nonincluded_companyid + nonincluded_compid_day ) )
    pickle.dump(nonincluded_companyid, open('comp_ex.'+industry, 'wb'))
    
    # 2-2) in_companies 추가하기
    in_companies = in_companies.append(in_companies_day).drop_duplicates()
    pickle.dump(in_companies, open('comp_in.'+industry, 'wb'))
    
    # 2-3) in_revisits_day : "temp variable" == 회사명 기준 결제정보 필터링하기
    
    in_revisits_day = pd.merge(revisit_day, in_companies, how='inner', on='COMPANY_ID') # category info
    
    # 3) all_userid_day : temp variable
    # new userid만 찾아내기 위해 존재한다
    all_userid_day = list ( set(in_revisits_day.EPOPCON_ID.unique().tolist()) 
                      - set(included_userid + nonincluded_userid) )
    
    #try-except
    try:
        #3-1) users_day  : temp variable
        users_day = get_ci_adid(all_userid_day)

        #3-2) filtering : ad_id exists : users_day ->  included or not  
        in_users =  in_users.append(users_day[users_day.ADVTM_ID.notnull()]).drop_duplicates()
        pickle.dump(in_users, open('user_in.'+industry, 'wb'))

        nonincluded_userid = list(set(nonincluded_userid+users_day[users_day.ADVTM_ID.isnull()].EPOPCON_ID.unique().tolist()))
        pickle.dump(nonincluded_userid, open('user_ex.'+industry, 'wb'))
    except:
        print(d, 'no adid')
        pickle.dump(d + timedelta(days=1), open('indexDt.'+industry, 'wb'))
        continue
    
    #4) inserting merged-data to mezzo_car/beauty database
    revisit_in_dt_df_write(pd.merge(in_revisits_day, in_users, how='inner', on='EPOPCON_ID'), industry) # ad_id
        
    #5) looping 
    pickle.dump(day + timedelta(days=1), open('indexDt.'+industry, 'wb'))
    batch_day_count -= 1
    
    #5-1) completed messages
    print(len(revisit_day), 'pay records at', len(revisit_day.COMPANY_ID.unique().tolist()), 'stores.', len(compid_day), 'stores newly found,', len(in_revisits_day), 'beauty payments.')
    print(len(all_userid), 'total users,', len(in_users), 'total beauty users,' )
    print( len(included_userid), len(included_companyid))
    
    #6) end of looping
    if(batch_day_count <= 0 or index >= len(dates)-1):
        print('=========================================\n the process has done',datetime.now()-start_time)
        break
        
#print(a.shape, a.head())

=======processing : 3 %
new revisit_in pickle
new all user list pickle
new industry in user df pickle
new industry ex user list pickle
new industry in comp df pickle
new industry ex comp list pickle
get_ci_revisit_regDt :  0:00:27.368121 563274 revisit records.
get_ci_company_by_id :  0:01:28.391108 159482 company ids.
get_ci_adid :  0:00:01.652395
563274 pay records at 160294 stores. 160294 stores newly found, 19519 beauty payments.
277717 total users, 16589 total beauty users,
0 0
=======processing : 5 %
new revisit_in pickle
get_ci_revisit_regDt :  0:00:24.685680 561385 revisit records.
get_ci_company_by_id :  0:00:31.462686 85067 company ids.
get_ci_adid :  0:00:01.440831
561385 pay records at 158618 stores. 85860 stores newly found, 20017 beauty payments.
392335 total users, 31873 total beauty users,
16589 8067
=======processing : 8 %
new revisit_in pickle
get_ci_revisit_regDt :  0:00:28.532869 593539 revisit records.
get_ci_company_by_id :  0:00:19.650265 64590 company ids.
get_c

# INSERT BRAND_NAME TO MEZZO DB

In [22]:
xl_file = pd.read_excel('BRAND_PATTERN.xlsx',sheetname='Sheet1')
print(xl_file.shape)
xl_file.head()

(269, 2)


,co_name,pattern
0,웰메이드,웰메이드
1,힐앤힐,힐앤힐
2,흄,^흄
3,휴먼피부과,휴먼피부과
4,휠라,필라|휠라|FILA


In [23]:
import pymysql
# autocommit = true
start_time=datetime.now()

conn = pymysql.connect(host='192.168.0.50', user='eums', password='eums00!q', db='eums-shared2', charset='utf8', autocommit=True)
curs = conn.cursor(pymysql.cursors.DictCursor)

for i in range(len(xl_file.index)) : 
    queryString = "UPDATE mezzo_beauty SET BRAND_NAME = '"+xl_file.iloc[i,0]+"' WHERE REPLACE(co_name,'(주)','') REGEXP '"+xl_file.iloc[i,1].replace(" ", "")+"';"
    curs.execute(queryString)

curs.close()
conn.close()

print(datetime.now()-start_time)

0:04:08.340944


### Execute BRAND_COSMETICS.sql  (using MySQLdb later)

# EXPORT DB TO CSV FILE

In [3]:
from ci_db import read_mezzo,export_to_db
start_time=datetime.now()
# select * from mezzo_car( or mezzo_baeuty)
#car_df=read_mezzo ('car')
beauty_df=read_mezzo ('beauty')

In [4]:
# redefining orders and column names
#export_c = export_to_db(car_df)
export_b = export_to_db(beauty_df) # 할때마다 export_to_db() 수정필요-> 주석처리

#sample_c=export_c.iloc[:100]
sample_b=export_b.iloc[:100]


#export_c.to_csv('car.csv')
export_b.to_csv('beauty0302.csv')

#export_b.DEAL_DT.max()
print(datetime.now()-start_time)
sample_b.head()

empty 'ADVTM_ID' 11
2018-03-02 17:32:17.761585
2018-03-02 17:32:19.861170
0:00:57.054708


,DEAL_DT,ADID,BRAND_NAME,CO_NAME,CATE,CATE1,CARD_NAME,CARD_COMP,TAG,PAYMENT,PAY_COUNT
0,2018-01-27 23:29:04,adec4fcf-7e3c-4a23-bd8c-22f85629a273,올리브영,올리브영 태평로1가점,뷰티,화장품,삼성카드(0529),None,"#생활, #미용, #가정, #화장품, #올리브영",10000,236
1,2018-02-13 14:14:30,adec4fcf-7e3c-4a23-bd8c-22f85629a273,올리브영,올리브영 시청역점,뷰티,화장품,신한카드(9*5*),None,"#쇼핑,#유통,올리브영,RE,드럭스토어,#드럭스토어,#생활,#올리브영,#가정",5000,236
2,2018-02-06 23:09:21,adec4fcf-7e3c-4a23-bd8c-22f85629a273,올리브영,올리브영 시청역점,뷰티,화장품,신한카드(9*5*),None,"#쇼핑,#유통,올리브영,RE,드럭스토어,#드럭스토어,#생활,#올리브영,#가정",5000,236
3,2018-02-13 13:09:12,adec4fcf-7e3c-4a23-bd8c-22f85629a273,올리브영,올리브영 태평로1가점,뷰티,화장품,삼성카드(0529),None,"#생활, #미용, #가정, #화장품, #올리브영",5000,236
4,2018-01-27 09:17:57,adec4fcf-7e3c-4a23-bd8c-22f85629a273,올리브영,올리브영 태평로1가점,뷰티,화장품,삼성카드(0529),None,"#생활, #미용, #가정, #화장품, #올리브영",-5000,236


# Check csv file 

import pandas as pd
data = pd.read_csv('beauty0302.csv')
print(data.shape)
data.head()

In [11]:
print(beauty_df.shape)
beauty_df[[ 'DEAL_DT', 'ADVTM_ID', 'BRAND_NAME', 'CO_NAME', 'CATE', 'CATE1', 'CARD_NAME', 'CARD_COMP', 'TAG', 'PAYMENT' ]].tail(15)

(924048, 16)


,DEAL_DT,ADVTM_ID,BRAND_NAME,CO_NAME,CATE,CATE1,CARD_NAME,CARD_COMP,TAG,PAYMENT
924033,2018-02-28 23:52:57,2f8334ce-e011-46dd-b48e-00bc2d231bcd,에뛰드하우스,에뛰드하우스 이마트연제점,뷰티,화장품,하나카드(7*2*),None,"#쇼핑, #향수, #유통, #화장품",13000
924034,2018-02-28 23:53:24,76de4097-7c3b-4d1e-a448-11fcd265f445,None,바니스타일,패션,의류,삼성카드(0432),None,"#패션, #생활, #가정, #의류판매",27200
924035,2018-02-28 23:53:23,73c270d1-533b-47d9-92f5-9b831ac5886b,이니스프리,이니스프리 수원화서점,뷰티,화장품,삼성카드(1184),None,"#화장품, #생활, #미용, #가정, #이니스프리",49700
924036,2018-02-28 23:53:52,fbf86a78-c468-4f6a-8ef2-45b8b23158e4,더풋샵,더풋샵 성신여대점,뷰티,마사지,농협카드,None,"#더풋샵,#발관리,#생활,#미용,#가정",11000
924037,2018-02-28 23:54:33,4f2bfaf6-fee3-412f-8490-234784031237,None,롯데,뷰티,마사지,하나카드(9*1*),None,"#피부관리, #생활, #미용, #가정",-1799000
924038,2018-02-28 23:55:05,e5a8060f-faa4-48f3-a838-6d4b26d163fb,None,더블유,뷰티,화장품,기업BC(0871),None,"#쇼핑, #향수, #미용, #유통, #화장품",126960
924039,2018-02-28 23:55:36,a5afb94d-e864-461e-abf2-27c034852bbf,프라다,프라다 롯데호텔면세점,패션,의류,농협BC(3069),None,"#쇼핑,#유통,#종합패션,#패션,#생활,#의류판매,#가정",95000
924040,2018-02-28 23:56:09,8aaa3f85-60f9-4cf0-a75e-06a030f2a20a,None,진케이스,패션,보석,SC은행BC(9152),None,"#귀금속, #보석, #패션, #생활, #가정",32000
924041,2018-02-28 23:57:22,05bdabf9-fe82-425c-8b36-b6edb8fa0522,None,스킨 셀레네,뷰티,마사지,삼성카드(8909),None,"#마사지, #미용, #뷰티, #피부, #체형관리, #피부미용",32000
924042,2018-02-28 23:57:17,3b4ce5d7-3b29-46fb-a999-d4677e104d3c,None,1번가,패션,액세서리,우리카드(1784),None,"#쇼핑,#유통,#귀금속,#시계",23500


In [19]:
emptyADID= beauty_df[beauty_df['ADVTM_ID']=='']
print(emptyADID.shape)
emptyADID.head()

(11, 16)


,ADVTM_ID,BRAND_NAME,CARD_COMP,CARD_NAME,CATE,CATE1,COMPANY_ID,CO_NAME,CO_NAME_R,DEAL_DT,EPOPCON_ID,ID,ISSUED_COMPANY_ID,PAYMENT,REG_DT,TAG
62681,,올리브영,None,신한체크(2*3*),뷰티,화장품,5404034,올리브영 시흥대야점,올리브영시흥대야점,2018-01-26 12:50:59,eppcn15a2e18768c0c245990ee4d61,62719,2,22600,2018-01-26 12:51:05,"올리브영,RE,드럭스토어,#드럭스토어,#생활,#올리브영,#가정"
438611,,유니클로,None,우리카드(4700),패션,의류,743036,유니클로 대구동성로중앙점,유니클로대구동성로중앙점,2018-01-25 15:11:16,eppcn15a7ecb3a8c0c243d38b36fc5,438722,6,152500,2018-02-10 19:37:24,"#패션, #유니클로, #생활, #의류판매, #가정"
497260,,None,None,우리카드(4700),패션,의류,6811287,(주)클랍핸즈스토어,클랍핸즈스토어,2018-02-12 18:41:02,eppcn15a7ecb3a8c0c243d38b36fc5,497375,6,31200,2018-02-12 18:42:19,"#패션, #생활, #가정, #의류판매"
601308,,None,None,신한카드(1*9*),뷰티,헤어,6658272,샤넬헤어케어,샤넬헤어케어,2018-01-29 18:06:03,eppcn15a86358a8c0c243d38b3715c,601436,2,30000,2018-02-16 10:37:36,"#생활, #미용실, #미용, #가정"
601309,,None,None,롯데카드(4*0*),패션,의류,6961247,더스토리,더스토리,2018-02-10 23:01:16,eppcn15a86358a8c0c243d38b3715c,601437,10,140000,2018-02-16 10:37:36,"#패션, #생활, #가정, #의류판매"


In [13]:
print(export_b.shape)
export_b[[ 'DEAL_DT', 'ADID', 'BRAND_NAME', 'CO_NAME', 'CATE', 'CATE1', 'CARD_NAME', 'CARD_COMP', 'TAG', 'PAYMENT' ]].tail()

(924037, 11)


,DEAL_DT,ADID,BRAND_NAME,CO_NAME,CATE,CATE1,CARD_NAME,CARD_COMP,TAG,PAYMENT
924032,2018-02-24 13:07:51,2372e359-1a2e-400b-b4f4-d9571ef56f61,None,웰빙앤스파,뷰티,마사지,현대카드,None,"#피부관리, #생활, #미용, #가정",40000
924033,2018-02-12 12:32:35,dd28bfab-3fed-493b-bd39-1ba35eaa4a20,올리브영,올리브영 신정네거리역점,뷰티,화장품,신한체크(7*9*),None,"올리브영,RE,드럭스토어,#드럭스토어,#생활,#올리브영,#가정",8600
924034,2018-02-24 12:21:13,7b029c41-9a99-4bcc-9b6a-79928302221a,올리브영,올리브영 씨지브이북수원점,뷰티,화장품,삼성카드(8447),None,"#생활, #미용, #가정, #화장품, #올리브영",-50000
924035,2018-02-24 13:46:15,bc4272cd-697d-4c08-93ee-adc45be093c2,올리브영,올리브영 씨지브이북수원점,뷰티,화장품,우리카드(6924),None,"#생활, #미용, #가정, #화장품, #올리브영",6000
924036,2018-02-06 15:07:05,0d4bf939-e0cd-4add-83b1-55640b9d3367,애터미,애터미하우스,뷰티,화장품,신한체크(8*4*),None,"#기타,#화장품,#생활,#미용,#가정",46700


* 이틀치 했을때 5만개 정도 CSV DATA 만들어짐